In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
titanic_train = pd.read_csv("/kaggle/input/titanic/train.csv", index_col='PassengerId')
titanic_test = pd.read_csv("/kaggle/input/titanic/test.csv", index_col = "PassengerId")

train_results = titanic_train["Survived"].copy()
titanic_train.drop("Survived", axis=1, inplace=True, errors="ignore")
titanic = pd.concat([titanic_train, titanic_test])
traindex = titanic_train.index
testdex = titanic_test.index

In [3]:
#preliminary explorati  
titanic["Gen"] = np.ones((titanic.shape[0]))
titanic.Gen.loc[titanic.Sex.isin(['female'])] = 0

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [4]:
train_results.head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

In [5]:
titanic.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Gen
PassengerId,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1.0
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0.0
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0.0
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0.0
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1.0


In [6]:
#Cabin Filling
cabin_only = titanic[["Cabin"]].copy()
cabin_only["Cabin_Data"] = cabin_only["Cabin"].isnull().apply(lambda x: not x) #find cabins with data

cabin_only["Deck"] = cabin_only["Cabin"].str.slice(0,1)
cabin_only["Room"] = cabin_only["Cabin"].str.slice(1,5).str.extract(("([0-9]+)"), expand = False).astype(float)
cabin_only[cabin_only["Cabin_Data"]]

cabin_only.drop(["Cabin","Cabin_data"], axis = 1, inplace = True, errors="ignore")
cabin_only["Deck"] = cabin_only["Deck"].fillna("N")
cabin_only["Room"] = cabin_only["Room"].fillna(cabin_only["Room"].mean())

In [7]:
#ONE HOT ENCODING OF CABIN DATA
def one_hot_col(df, label, drop_col=False):
    '''
    This function will one hot encode the chosen column.
    Args:
        df: Pandas dataframe
        label: Label of the column to encode
        drop_col: boolean to decide if the chosen column should be dropped
    Returns:
        pandas dataframe with the given encoding
    '''
    one_hot = pd.get_dummies(df[label],prefix=label) #retreive dummy vars
    if drop_col: #drop column if desired
        df=df.drop(label,axis=1) 
    df = df.join(one_hot) #join the dummy variables to datafram
    return df

def one_hot(df, labels, drop_col=False):
    '''
    This function will one hot encode a list of columns.
    Args:
        df: Pandas dataframe
        labels: list of the columns to encode
        drop_col: boolean to decide if the chosen column should be dropped
    Returns:
        pandas dataframe with the given encoding
    '''
    for label in labels:
        df = one_hot_col(df, label, drop_col)
    return df

cabin_only = one_hot(cabin_only, ["Deck"], drop_col = True)


In [8]:
cabin_only.head()

,Cabin_Data,Room,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_N,Deck_T
PassengerId,,,,,,,,,,,
1,False,49.615917,0,0,0,0,0,0,0,1,0
2,True,85.000000,0,0,1,0,0,0,0,0,0
3,False,49.615917,0,0,0,0,0,0,0,1,0
4,True,123.000000,0,0,1,0,0,0,0,0,0
5,False,49.615917,0,0,0,0,0,0,0,1,0


In [9]:
#Explore relationship between vars
for column in cabin_only.columns.values[1:]: #append the dummy vars
    titanic[column] = cabin_only[column]
titanic[column] = titanic.drop(['Ticket','Cabin'], axis = 1, inplace = True)
corr = titanic.corr()

In [10]:
corr["Pclass"]

Pclass    1.000000
Age      -0.408106
SibSp     0.060832
Parch     0.018322
Fare     -0.558629
Gen       0.124617
Room     -0.018452
Deck_A   -0.202143
Deck_B   -0.353414
Deck_C   -0.430044
Deck_D   -0.265341
Deck_E   -0.225649
Deck_F    0.013122
Deck_G    0.052133
Deck_N    0.713857
Name: Pclass, dtype: float64

In [11]:
corr["Fare"].sort_values(ascending=False)

Fare      1.000000
Deck_C    0.401382
Deck_B    0.393751
Parch     0.221539
Age       0.178740
SibSp     0.160238
Room      0.077825
Deck_E    0.073950
Deck_D    0.072738
Deck_A    0.020095
Deck_G   -0.022857
Deck_F   -0.037567
Gen      -0.185523
Deck_N   -0.507253
Pclass   -0.558629
Name: Fare, dtype: float64

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import scipy.stats as st

In [13]:
#append other variables
more_vars = cabin_only.join(titanic.Gen).join(titanic.SibSp).join(titanic.Parch).join(titanic.Pclass)

In [14]:
# Train
train_df = more_vars.loc[traindex, :]
print(train_df)
train_df['Survived'] = train_results


#features = ['Pclass','Sex','SibSp','Parch'] #did not include Cabin, which is mostly NaN
X = train_df.drop("Survived",axis=1).copy()
X_test = more_vars.loc[testdex, :]

y = train_df['Survived']

             Cabin_Data        Room  Deck_A  Deck_B  Deck_C  Deck_D  Deck_E  \
PassengerId                                                                   
1                 False   49.615917       0       0       0       0       0   
2                  True   85.000000       0       0       1       0       0   
3                 False   49.615917       0       0       0       0       0   
4                  True  123.000000       0       0       1       0       0   
5                 False   49.615917       0       0       0       0       0   
...                 ...         ...     ...     ...     ...     ...     ...   
887               False   49.615917       0       0       0       0       0   
888                True   42.000000       0       1       0       0       0   
889               False   49.615917       0       0       0       0       0   
890                True  148.000000       0       0       1       0       0   
891               False   49.615917       0       0 

In [15]:

for_model = RandomForestClassifier(random_state=1)

for_model.fit(X,y)

predictions = for_model.predict(X_test)
'''
rfc = RandomForestClassifier()

param_grid ={'max_depth': st.randint(6, 11),
             'n_estimators':st.randint(300, 500),
             'max_features':np.arange(0.5,.81, 0.05),
            'max_leaf_nodes':st.randint(6, 10)}

grid = RandomizedSearchCV(rfc,
                    param_grid, cv=10,
                    scoring='accuracy',
                    verbose=1,n_iter=20)

grid.fit(X, y)'''

"\nrfc = RandomForestClassifier()\n\nparam_grid ={'max_depth': st.randint(6, 11),\n             'n_estimators':st.randint(300, 500),\n             'max_features':np.arange(0.5,.81, 0.05),\n            'max_leaf_nodes':st.randint(6, 10)}\n\ngrid = RandomizedSearchCV(rfc,\n                    param_grid, cv=10,\n                    scoring='accuracy',\n                    verbose=1,n_iter=20)\n\ngrid.fit(X, y)"

In [16]:
#grid.best_estimator
#grid.best_score_
#predictions = grid.best_estimator_.predict(test_df)

In [17]:
#results_df = pd.DataFrame()
#results_df["PassngerId"] = test_df.index
#results_df["Predictions"] = predictions

In [18]:
#results_df.to_csv("Predictions", index=False)

In [19]:

output = pd.DataFrame({'PassengerId': X_test.index, 'Survived': predictions})
output.to_csv('my_sub.csv', index=False)
print("Your submission was successfully saved!")


Your submission was successfully saved!
